<a href="https://colab.research.google.com/github/Jinops/ie-algo/blob/main/ch9_pebble_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 9장. 조약돌 풀이 #



팩토리얼

In [ ]:
def facto(n):
    if n==1:
        return 1
    return n * facto(n-1)

facto(3)

6

피보 (재귀)

In [ ]:
def fibo(n):
    if n==1 or n==2:
        return 1
    return fibo(n-1)+fibo(n-2)
# 1 1 2 3 5 8
fibo(6)

8

피보 (dp: bottom-up)

In [ ]:
def fibo_bu(n):
    dp=[0]*(n+1)
    dp[1] = 1
    dp[2] = 1
    for i in range(3, n+1):
        dp[i] = dp[i-1]+dp[i-2]
    return dp[n]

피보 (dp: top-down)

In [ ]:
def fibo_td(n, dp=[]):
    if len(dp)==0:
        dp = [0]*(n+1)
        dp[1] = 1
        dp[2] = 1
    if dp[n] != 0:
        return dp[n]
    else:
        dp[n] = fibo_td(n-1, dp) + fibo_td(n-2, dp)
    return dp[n]

시간 비교

In [ ]:
import time

start_time = time.time()
print("facto: ", facto(6))
print("%.4f sec." % (time.time() - start_time))

start_time = time.time()
print("fibo: ", fibo(20))
print("%.4f sec." % (time.time() - start_time))

start_time = time.time()
print("fibo (bottom_up): ", fibo_bu(20))
print("%.4f sec." % (time.time() - start_time))

start_time = time.time()
print("fibo (top_down): ", fibo_td(20))
print("%.4f sec." % (time.time() - start_time))

facto:  720
0.0021 sec.
fibo:  6765
0.0052 sec.
fibo (bottom_up):  6765
0.0002 sec.
fibo (top_down):  6765
0.0002 sec.


**(1) weight 계산하기 (두 가지 방법)**



In [ ]:
A1 = [[6,-8,11], [7,10,12],[12,14,7],[-5, 9, 4],[5, 7, 8],[3, 13, -2],[11, 8, 9],[3, 5, 4]]
A2 = [[6,-8,11], [7,10,12],[12,14,7],[-5, 9, 4],[5, 7, 8],[3, 13, -2],[11, 8, 9],[3, 5, 4],
     [6,-8,11], [7,10,12],[12,14,7],[-5, 9, 4],[5, 7, 8],[3, 13, -2],[11, 8, 9],[3, 5, 4]]

def append_weight(A):
    w = []
    for i in range(len(A)):
        pattern0 = A[i][0] # o x x
        pattern1 = A[i][1] # x o x
        pattern2 = A[i][2] # x x o
        pattern3 = A[i][0] + A[i][2] # o x o
        w.append([pattern0, pattern1, pattern2, pattern3])
    return w

w1 = append_weight(A1)
w1

[[6, -8, 11, 17],
 [7, 10, 12, 19],
 [12, 14, 7, 19],
 [-5, 9, 4, -1],
 [5, 7, 8, 13],
 [3, 13, -2, 1],
 [11, 8, 9, 20],
 [3, 5, 4, 7]]

**(2) Pebble 재귀적 호출 버전**



In [ ]:
def pebbleSum():
    n = len(w)
    m = [pebble(n-1,0), pebble(n-1,1), pebble(n-1,2), pebble(n-1,3)]
    return max(m)

def pebble(i, p):
    if i == 0:
        return w[0][p]
    else:
        M = None
        for q in range(4):
            if compatible(p, q):
                tmp = pebble(i-1, q)
                if M == None or tmp > M:
                    M = tmp
        return w[i][p]+M

def compatible(p,q):
    if p==0:
        return q==1 or q==2
    if p==1:
        return q==0 or q==2 or q==3
    if p==2:
        return q==0 or q==1
    if p==3:
        return q==1
    return False

import time

start_time = time.time()
# w = append_weight(A1)
w = append_weight(A2)
print(pebbleSum())
print("\n %.4f sec." % (time.time() - start_time))

212

 0.5288 sec.


**(3) Pebble DP (top-down) 버전**

In [ ]:
def pebbleSum_DP_topdown():
  n = len(w)
  m = [pebble_DP_topdown(n-1,0), pebble_DP_topdown(n-1,1), pebble_DP_topdown(n-1,2), pebble_DP_topdown(n-1,3)]
  return max(m)

def pebble_DP_topdown(i, p):
  if i==0:
    peb[0][p] = w[0][p]
    return peb[0][p]

  prev = []
  for q in range(4):
    if compatible(p,q):
      if peb[i-1][q] == None:
        pebble_DP_topdown(i-1, q)
      prev.append(peb[i-1][q])

  peb[i][p] = w[i][p] + max(prev)
  return peb[i][p]

start_time = time.time()
# w = append_weight(A1)
w = append_weight(A2)
peb = [[None] * len(w[0]) for _ in range(len(w))]
print(pebbleSum_DP_topdown())
print("\n %.4f sec." % (time.time() - start_time))

212

 0.0010 sec.


**(4) Pebble DP (bottom-up) 버전**

In [ ]:
def pebbleSum_DP_bottomup():
  for p in range(4):
    peb[0][p] = w[0][p]
  for i in range(1, len(w)):
    for p in range(4):
      prev=[]
      for q in range(4):
        if compatible(p,q):
          prev.append(peb[i-1][q])
      peb[i][p] = w[i][p] + max(prev)
  return max(peb[len(w)-1])

start_time = time.time()
w = append_weight(A2)
peb = [[None] * len(w[0]) for _ in range(len(w))]
print(pebbleSum_DP_bottomup())
print("\n %.4f sec." % (time.time() - start_time))

212

 0.0006 sec.
